In [1]:
import os
import gc
import time
import pickle
import numpy as np
import pandas as pd
import math
from collections import defaultdict
from sklearn.cross_validation import train_test_split
#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False

/usr/local/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/miniconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
cache_path = 'fuck_copy4/'
train_path = '../input/训练数据-ccf_first_round_user_shop_behavior.csv'
shop_path='../input/训练数据-ccf_first_round_shop_info.csv'
test_path = '../input/AB榜测试集-evaluation_public.csv'
wifi_shop_info=pd.DataFrame()
wifi_shop_max=pd.DataFrame()
wifi_shop_connect_only=pd.DataFrame()
shop_loc=pd.DataFrame()
wifi_shop_count=pd.DataFrame()
flag = True

In [ ]:
train_feat = pd.read_csv('fuck_copy_temp/train_feat_fuck4_add10.csv')
test_feat = pd.read_csv('fuck_copy_temp/test_feat_fuck4_add10.csv')

In [4]:
import xgboost as xgb
predictors = ['h_s','time_dif','h_s_v','h_s_m','knn_max_values','wifi_count_rt','wifishopcount','h3','knn_values','longitude_s', 'latitude_s',
    'real_lon_dif','real_lat_dif','w3','lon_real','lat_real','real_ang_df','real_dis_df','connect_sum',
    'ang_df','dis_df','mall_id','user_id','dayofweek','hourofday','connect1','connect2','power_dif2','ave_power2','power_dif1',
    'ave_power1','user_kind_count','category','price','user_shop_count',
   'shop_hot','user_count',
    'ave_power3','ave_power4','ave_power5',
    'ave_power6','ave_power7','ave_power8','ave_power9','ave_power10',
    'max_power1','max_power2','max_power3','max_power4','max_power5','max_power6','max_power7','max_power8','max_power9','max_power10',
   'wifi inshopcount1','wifi inshopcount2','wifi inshopcount3','wifi inshopcount4','wifi inshopcount5',
    'wifi inshopcount6','wifi inshopcount7','wifi inshopcount8','wifi inshopcount9','wifi inshopcount10',
         'min_power1','min_power2','min_power3','min_power4','min_power5','min_power6','min_power7','min_power8','min_power9','min_power10',
             ]
params = {
        'objective': 'binary:logistic',
        'eta': 0.05,
        'colsample_bytree': 0.886,
        'min_child_weight': 1.1,
        'max_depth': 7,
        'subsample': 0.886,
        'gamma': 0.1,
        'lambda':10,
        'verbose_eval': True,
        'eval_metric': 'auc',
        'scale_pos_weight': 10,
        'seed': 201703,
        'missing':-1
    }
print ('start training')
gc.collect()
x_train, x_val, y_train, y_val = train_test_split(train_feat[predictors], train_feat['label'], test_size=0.3,random_state=1)
dval = xgb.DMatrix(x_val,label=y_val)
dtrain = xgb.DMatrix(x_train, label=y_train)
xgbtrain = xgb.DMatrix(train_feat[predictors], train_feat['label'])
xgbtest = xgb.DMatrix(test_feat[predictors])
print ('start training')
watchlist  = [(dtrain,'train'),(dval,'val')]

start training


0

start training


In [ ]:
print ('start training')
rounds = 1600
model = xgb.train(params, xgbtrain, rounds)
gc.collect()
print ('start predicting')
test_feat.loc[:,'pred'] = model.predict(xgbtest)
test_feat = test_feat[['row_id','shop_id','pred']].drop_duplicates()
gc.collect()
result=test_feat.groupby('row_id',as_index=False)['pred'].agg({'pred':'max'})
result.to_csv('fuck_submit_before/result_fuck4_ave_inshopcount_max_min.csv',index=False)
test_feat.to_csv('fuck_submit_before/test_feat_fuck4_ave_inshopcount_max_min.csv',index=False)

start training


In [3]:
result = pd.read_csv('fuck_submit_before/result_fuck4_ave10_inshopcount10_difave_difmax_Min.csv')
test_feat = pd.read_csv('fuck_submit_before/test_feat_fuck4_ave10_inshopcount10_difave_difmax_Min.csv')
result.head()
test_feat.head()

,row_id,pred
0,1,0.999772
1,2,0.999744
2,3,0.995313
3,4,0.985377
4,5,0.920314


,row_id,shop_id,pred
0,432969,s_11611,0.006421
1,432969,s_497319,0.977793
2,53780,s_137744,0.001581
3,53780,s_3392998,0.003020
4,53780,s_350316,0.967899


In [ ]:
test_feat.shape
result.shape

In [10]:
result=pd.merge(result,test_feat,on=['row_id','pred'],how='left')
test = pd.read_csv(test_path)
result = pd.merge(test[['row_id']],result[['row_id','shop_id']],on='row_id',how='left')
result.fillna('0',inplace=True)
result.to_csv('result.csv',index=False)